In [20]:
import pandas as pd
import yaml
from collections import Counter
from pathlib import Path
from sklearn.model_selection import KFold
from ultralytics import YOLO
import os
import glob
import shutil

In [21]:
TARGET_IMAGES_PATH = './images/'
TARGET_LABELS_PATH = './labels/'

image_paths = glob.glob(TARGET_IMAGES_PATH + "*.jpg")
label_paths = glob.glob(TARGET_LABELS_PATH + "*.txt")

print(image_paths)
print(label_paths)


# source_folder_path = "./axial/cross_data/{split}/{type}"
# splits = ["train", "valid"]
# types = ["images/*.jpg", "labels/*.txt"]

# os.makedirs(TARGET_IMAGES_PATH, exist_ok=True)
# os.makedirs(TARGET_LABELS_PATH, exist_ok=True)

# image_paths = list()
# label_paths = list()

# for split in splits:
#     for data_type in types:
#         files = glob.glob(source_folder_path.format(split=split, type=data_type))
#         for file_ in files:
#             if "image" in data_type:
#                 shutil.copy(file_, TARGET_IMAGES_PATH)
#             else:
#                 shutil.copy(file_, TARGET_LABELS_PATH)

# # Store image and label paths for future use
# image_paths = glob.glob(TARGET_IMAGES_PATH + "*.jpg")
# label_paths = glob.glob(TARGET_LABELS_PATH + "*.txt")

# print(f"Copied {len(image_paths)} images to {TARGET_IMAGES_PATH}")
# print(f"Copied {len(label_paths)} labels to {TARGET_LABELS_PATH}")

['./images/89 (5).jpg', './images/67 (5).jpg', './images/104.jpg', './images/87.jpg', './images/64 (8).jpg', './images/63 (2).jpg', './images/93 (6).jpg', './images/71.jpg', './images/81 (4).jpg', './images/89.jpg', './images/62 (4).jpg', './images/79 (6).jpg', './images/67 (2).jpg', './images/71 (5).jpg', './images/108.jpg', './images/106.jpg', './images/93 (4).jpg', './images/92 (6).jpg', './images/80.jpg', './images/71 (3).jpg', './images/72 (4).jpg', './images/78 (4).jpg', './images/70.jpg', './images/64.jpg', './images/70 (6).jpg', './images/65 (6).jpg', './images/76 (3).jpg', './images/95 (2).jpg', './images/64 (7).jpg', './images/106 (2).jpg', './images/69 (3).jpg', './images/76 (10).jpg', './images/80 (6).jpg', './images/67 (8).jpg', './images/109.jpg', './images/84 (4).jpg', './images/73 (6).jpg', './images/74 (12).jpg', './images/75 (3).jpg', './images/102 (3).jpg', './images/70 (9).jpg', './images/72 (5).jpg', './images/62 (13).jpg', './images/62 (14).jpg', './images/62 (6).

In [22]:
dataset_path = Path("./")  # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'
labels

[PosixPath('labels/100 (2).txt'),
 PosixPath('labels/100 (3).txt'),
 PosixPath('labels/100.txt'),
 PosixPath('labels/102 (2).txt'),
 PosixPath('labels/102 (3).txt'),
 PosixPath('labels/102 (4).txt'),
 PosixPath('labels/102.txt'),
 PosixPath('labels/103 (2).txt'),
 PosixPath('labels/103.txt'),
 PosixPath('labels/104 (2).txt'),
 PosixPath('labels/104 (3).txt'),
 PosixPath('labels/104.txt'),
 PosixPath('labels/105.txt'),
 PosixPath('labels/106 (2).txt'),
 PosixPath('labels/106.txt'),
 PosixPath('labels/107.txt'),
 PosixPath('labels/108.txt'),
 PosixPath('labels/109 (2).txt'),
 PosixPath('labels/109.txt'),
 PosixPath('labels/110 (2).txt'),
 PosixPath('labels/110.txt'),
 PosixPath('labels/111.txt'),
 PosixPath('labels/112 (2).txt'),
 PosixPath('labels/112.txt'),
 PosixPath('labels/113 (2).txt'),
 PosixPath('labels/113.txt'),
 PosixPath('labels/114.txt'),
 PosixPath('labels/115.txt'),
 PosixPath('labels/116.txt'),
 PosixPath('labels/118.txt'),
 PosixPath('labels/119.txt'),
 PosixPath('labels

In [23]:
yaml_file = './no_val.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = list(range(len(classes)))
print(list(zip(classes, cls_idx)))

[('negative', 0), ('positive', 1)]


In [24]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)
labels_df

,0,1
100 (2),NaN,NaN
100 (3),NaN,NaN
100,NaN,NaN
102 (2),NaN,NaN
102 (3),NaN,NaN
...,...,...
98,NaN,NaN
99 (2),NaN,NaN
99 (3),NaN,NaN
99 (4),NaN,NaN


In [25]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`
labels_df

/tmp/ipykernel_37942/3131205890.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`


,0,1
100 (2),1.0,0.0
100 (3),0.0,1.0
100,1.0,0.0
102 (2),0.0,1.0
102 (3),0.0,1.0
...,...,...
98,1.0,0.0
99 (2),0.0,1.0
99 (3),0.0,1.0
99 (4),0.0,1.0


In [26]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [27]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [28]:
fold_lbl_distrb

,0,1
split_1,0.258993,0.218045
split_2,0.30597,0.191176
split_3,0.25,0.265625
split_4,0.206897,0.317073
split_5,0.232394,0.265625


In [29]:
import os
import shutil
import yaml
from pathlib import Path

kfold_base_path = Path('./kfold_coronal')
shutil.rmtree(kfold_base_path) if kfold_base_path.is_dir() else None  # 기존 폴더가 있으면 삭제
os.makedirs(str(kfold_base_path))  # 새 폴더 생성

yaml_paths = list()
train_txt_paths = list()
val_txt_paths = list()

# 이미지 및 라벨 파일 경로를 절대 경로로 변경
absolute_image_paths = [str(Path(p).absolute()) for p in image_paths]

for i, (train_idx, val_idx) in enumerate(kfolds):
    # Get image paths for train-val split
    train_paths = [absolute_image_paths[j] for j in train_idx]
    val_paths = [absolute_image_paths[j] for j in val_idx]
    
    # Create text files to store image paths
    train_txt = kfold_base_path / f"train_{i}.txt"
    val_txt = kfold_base_path / f"val_{i}.txt"
    
    # Write images paths for training and validation in split i
    with open(str(train_txt), 'w') as f:
        f.writelines(s + '\n' for s in train_paths)
    with open(str(val_txt), 'w') as f:
        f.writelines(s + '\n' for s in val_paths)
    
    train_txt_paths.append(str(train_txt))
    val_txt_paths.append(str(val_txt))
    
    # Create dataset yaml file
    dataset_yaml = kfold_base_path / f"data_{i}.yaml"
    yaml_content = {
        'path': str(kfold_base_path),  # Root path for dataset
        'train': str(train_txt),       # Train txt file path
        'val': str(val_txt),           # Validation txt file path
        'test': "/home/under1/Detect/jeongui/cross_val/coronal/yolo_data/images/test",
        'names': classes               # Class names
    }
    with open(dataset_yaml, 'w') as f:
        yaml.dump(yaml_content, f)
    
    yaml_paths.append(str(dataset_yaml))

# 디버깅을 위한 출력 추가
for i in range(len(kfolds)):
    print(f"Using dataset YAML: {yaml_paths[i]}")
    with open(yaml_paths[i], 'r') as f:
        print(f.read())
    print(f"Train file contents: {train_txt_paths[i]}")
    with open(train_txt_paths[i], 'r') as f:
        print(f.read())
    print(f"Val file contents: {val_txt_paths[i]}")
    with open(val_txt_paths[i], 'r') as f:
        print(f.read())


Using dataset YAML: kfold_coronal/data_0.yaml
names:
- negative
- positive
path: kfold_coronal
test: /home/under1/Detect/jeongui/cross_val/coronal/yolo_data/images/test
train: kfold_coronal/train_0.txt
val: kfold_coronal/val_0.txt

Train file contents: kfold_coronal/train_0.txt
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/67 (5).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/104.jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/87.jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/64 (8).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/63 (2).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/93 (6).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/89.jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/62 (4).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/79 (6).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/

In [31]:
print("data_0.yaml File: \n")
!cat ./kfold_coronal/data_0.yaml

print("\ntrain_0.txt first two lines: \n")
!cat ./kfold_coronal/train_0.txt | head -n 2


data_0.yaml File: 

names:
- negative
- positive
path: kfold_coronal
test: /home/under1/Detect/jeongui/cross_val/coronal/yolo_data/images/test
train: kfold_coronal/train_0.txt
val: kfold_coronal/val_0.txt

train_0.txt first two lines: 

/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/67 (5).jpg
/home/under1/Detect/jeongui/cross_val/coronal/cross_data/images/104.jpg


In [33]:
from IPython.display import clear_output
import time

batch = 16
project = 'kfold_demo'
epochs = 200

results = list()

for i in range(ksplit):
    model = YOLO('yolov8n.pt')
    dataset_yaml =yaml_paths[i]
    print(f"Training for fold={i} using {dataset_yaml}")
    model.train(data=dataset_yaml, batch=batch, project=project, epochs=epochs, verbose=False, workers=28)
    result = model.metrics # Metrics on validation set
    results.append(result) # save output metrics for further analysis
    clear_output()

In [34]:
metric_values = dict()

for result in results:
    for metric, metric_val in result.results_dict.items():
        if metric not in metric_values:
            metric_values[metric] = []
        metric_values[metric].append(metric_val)

metric_df = pd.DataFrame.from_dict(metric_values)
visualize_metric = ['mean', 'std', 'min', 'max']
metric_df.describe().loc[visualize_metric]

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),fitness
mean,0.854786,0.833502,0.884240,0.682168,0.702375
std,0.045014,0.043092,0.028053,0.028314,0.028071
min,0.792318,0.771225,0.856188,0.666355,0.685338
max,0.915211,0.874277,0.928861,0.732687,0.752304


In [35]:
metric_values

{'metrics/precision(B)': [0.8367545587545587,
  0.7923184844036492,
  0.8683686568837103,
  0.9152107745360948,
  0.8612799215576994],
 'metrics/recall(B)': [0.8079044117647058,
  0.849238032132769,
  0.8742765299559143,
  0.8648669545562201,
  0.7712253424058979],
 'metrics/mAP50(B)': [0.856188429983229,
  0.8654685754209979,
  0.8880986718626305,
  0.9288611955478623,
  0.8825821862244574],
 'metrics/mAP50-95(B)': [0.6663548948351457,
  0.6701123187456248,
  0.6719720450805678,
  0.7326869578057389,
  0.6697121354054033],
 'fitness': [0.685338248349954,
  0.6896479444131621,
  0.6935847077587741,
  0.7523043815799513,
  0.6909991404873087]}